In [3]:
pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/a8/e9/3e4879974a7c4dcaca2a746dde3df08d0ae8f14c74b03591616ce5f0a8b1/scikit_learn-1.4.0-1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for numpy<2.0,>=1.19.5 from https://files.pythonhosted.org/packages/3f/6b/5610004206cf7f8e7ad91c5a85a8c71b2f2f8051a0c0c4d5916b76d6cbb2/numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     ------------------------- ------------ 41.0/61.0 kB 487.6 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 651.3 kB/s eta 0:00:00
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/9a/25/5b30cb3efc9566f0ebeaeca1976150316353c17031ad7868ef46de5ab8dc/scipy-1.12.0-cp311-cp311-win_amd64.whl.metadata
     ----------------------------------------


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\MSI-1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:

import os
import shutil
from sklearn.model_selection import train_test_split

def split_dataset(dataset_dir, train_dir, val_dir, test_dir, test_size=0.15, val_size=0.15, random_state=42):
    # Create directories if they don't exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Get list of image files
    image_files = [f for f in os.listdir(dataset_dir) if f.endswith('.jpg')]

    # Split data into train, val, test sets
    train_val_files, test_files = train_test_split(image_files, test_size=test_size, random_state=random_state)
    train_files, val_files = train_test_split(train_val_files, test_size=val_size/(1-test_size), random_state=random_state)

    # Move files to respective directories
    move_files(train_files, dataset_dir, train_dir)
    move_files(val_files, dataset_dir, val_dir)
    move_files(test_files, dataset_dir, test_dir)

def move_files(files, src_dir, dst_dir):
    for file in files:
        shutil.move(os.path.join(src_dir, file), os.path.join(dst_dir, file))
        # Move corresponding annotation file if it exists
        annotation_file = file.replace('.jpg', '.txt')
        if os.path.exists(os.path.join(src_dir, annotation_file)):
            shutil.move(os.path.join(src_dir, annotation_file), os.path.join(dst_dir, annotation_file))

def update_yaml_config(train_dir, val_dir, test_dir):
    yaml_config = f"""
train: {train_dir}
val: {val_dir}
test: {test_dir}

nc: 1  # Number of classes
names: ['sports ball']  # Class names
"""
    with open('custom_data.yaml', 'w') as f:
        f.write(yaml_config)

# Define paths
dataset_dir = 'C:/robocon_24/cocoYOLO/downloaded_images/sports ball'
train_dir = 'C:/robocon_24/cocoYOLO/coco/train'
val_dir = 'C:/robocon_24/cocoYOLO/coco/val'
test_dir = 'C:/robocon_24/cocoYOLO/coco/test'

# Split dataset
split_dataset(dataset_dir, train_dir, val_dir, test_dir)

# Update YAML configuration file
update_yaml_config(train_dir, val_dir, test_dir)


In [2]:
pip install ultralytics


  Using cached ultralytics-8.1.10-py3-none-any.whl (709 kB)
     ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 MB 1.4 MB/s eta 0:00:02
     - -------------------------------------- 0.1/2.6 MB 1.3 MB/s eta 0:00:02
     --- ------------------------------------ 0.2/2.6 MB 1.4 MB/s eta 0:00:02
     ----- ---------------------------------- 0.4/2.6 MB 2.0 MB/s eta 0:00:02
     ---------- ----------------------------- 0.7/2.6 MB 2.8 MB/s eta 0:00:01
     ----------------- ---------------------- 1.1/2.6 MB 4.0 MB/s eta 0:00:01
     ------------------------------- -------- 2.0/2.6 MB 6.5 MB/s eta 0:00:01
     ---------------------------------------  2.6/2.6 MB 7.2 MB/s eta 0:00:01
     ---------------------------------------- 2.6/2.6 MB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     ---------------------------------------- 1.2/1.2 MB 37.3 MB/s eta 0:00:00
     -----------


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\MSI-1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data='C:/robocon_24/cocoYOLO/custom_data.yaml', epochs=10, imgsz=640)

100%|██████████| 6.23M/6.23M [00:00<00:00, 28.6MB/s]


Ultralytics YOLOv8.1.10 🚀 Python-3.10.11 torch-2.2.0+cpu CPU (AMD Ryzen 5 4500U with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/robocon_24/cocoYOLO/custom_data.yaml, epochs=10, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=

100%|██████████| 755k/755k [00:00<00:00, 10.8MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384, 128, 1]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  1     37248  ultralytics.nn.modules.block.C2f             [192, 64, 1]                  
 16                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 

...

      10/10         0G      1.119      1.102     0.9846          5        640: 100%|██████████| 187/187 [20:44<00:00,  6.65s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [01:10<00:00,  3.53s/it]

                   all        640        640      0.764      0.606      0.636      0.439



10 epochs completed in 3.652 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics YOLOv8.1.10 🚀 Python-3.10.11 torch-2.2.0+cpu CPU (AMD Ryzen 5 4500U with Radeon Graphics)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:58<00:00,  2.94s/it]


                   all        640        640      0.763      0.606      0.636      0.439
Speed: 2.4ms preprocess, 81.7ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\train


In [4]:
model_new = YOLO('runs/detect/train/weights/best.pt')

In [6]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\MSI-1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
pip install onnx

     ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/14.3 MB 1.3 MB/s eta 0:00:12
     ---------------------------------------- 0.1/14.3 MB 1.7 MB/s eta 0:00:09
      --------------------------------------- 0.3/14.3 MB 2.7 MB/s eta 0:00:06
     - -------------------------------------- 0.7/14.3 MB 4.2 MB/s eta 0:00:04
     --- ------------------------------------ 1.3/14.3 MB 6.5 MB/s eta 0:00:02
     ------ --------------------------------- 2.3/14.3 MB 9.7 MB/s eta 0:00:02
     --------- ------------------------------ 3.6/14.3 MB 12.7 MB/s eta 0:00:01
     -------------- ------------------------- 5.1/14.3 MB 15.6 MB/s eta 0:00:01
     ------------------ --------------------- 6.6/14.3 MB 17.7 MB/s eta 0:00:01
     ---------------------- ----------------- 8.1/14.3 MB 19.2 MB/s eta 0:00:01
     ------------------------ --------------- 8.9/14.3 MB 19.0 MB/s eta 0:00:01
     --------------------------- ------------ 9.7/14.3


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\MSI-1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
model_new.export(format = 'onnx')

Ultralytics YOLOv8.1.10 🚀 Python-3.10.11 torch-2.2.0+cpu CPU (AMD Ryzen 5 4500U with Radeon Graphics)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs\detect\train\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.9 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 1.7s, saved as 'runs\detect\train\weights\best.onnx' (11.7 MB)

Export complete (3.7s)
Results saved to C:\robocon_24\cocoYOLO\runs\detect\train\weights
Predict:         yolo predict task=detect model=runs\detect\train\weights\best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs\detect\train\weights\best.onnx imgsz=640 data=C:/robocon_24/cocoYOLO/custom_data.yaml  
Visualize:       https://netron.app


'runs\\detect\\train\\weights\\best.onnx'

In [4]:
import cv2

# Load ONNX Model
net = cv2.dnn.readNetFromONNX("runs/detect/train/weights/best.onnx")

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("Error opening video stream or file")
    exit()

# Read until video is completed
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret:
        # Feed frame to model
        net.setInput(cv2.dnn.blobFromImage(frame, size=(640, 640), swapRB=True, crop=False))
        outs = net.forward()

        # Debugging
        print("Model output shape:", [out.shape for out in outs])

        # Display frame
        cv2.imshow('Frame', frame)

        # Press Q on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

# When everything done, release the video capture object
cap.release()

# Close all windows
cv2.destroyAllWindows()


Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model output shape: [(5, 8400)]
Model ou

KeyboardInterrupt: 

: 

In [2]:
def predict(chosen_model, img, classes=[], conf=0.5):
    if classes:
        results = chosen_model.predict(img, classes=classes, conf=conf)
    else:
        results = chosen_model.predict(img, conf=conf)

    return results


def predict_and_detect(chosen_model, img, classes=[], conf=0.5):
    results = predict(chosen_model, img, classes, conf=conf)

    for result in results:
        for box in result.boxes:
            cv2.rectangle(img, (int(box.xyxy[0][0]), int(box.xyxy[0][1])),
                          (int(box.xyxy[0][2]), int(box.xyxy[0][3])), (255, 0, 0), 2)
            cv2.putText(img, f"{result.names[int(box.cls[0])]}",
                        (int(box.xyxy[0][0]), int(box.xyxy[0][1]) - 10),
                        cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 1)
    return img

In [3]:
image = cv2.imread("Videos/img.jpg")

result_img = predict_and_detect(model_new, image, classes=[], conf=0.5)

NameError: name 'cv2' is not defined

In [26]:
cv2.imshow("Image", result_img)

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - mat is not a numerical tuple
>  - Expected Ptr<cv::cuda::GpuMat> for argument 'mat'
>  - Expected Ptr<cv::UMat> for argument 'mat'


: 